In [ ]:
import os
import subprocess
from pathlib import Path
import json
from pydantic import BaseModel
import nest_asyncio
import shutil
import logfire
import csv

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.models.anthropic import AnthropicModel
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.models.groq import GroqModel

from great_tables import GT, style, loc
import pandas as pd

from dotenv import load_dotenv

logfire.configure()
nest_asyncio.apply()

result = load_dotenv(dotenv_path='llm_blue/.env')
print(f"load_dotenv result: {result}")


load_dotenv result: False


Logfire project URL: ]8;id=193222;https://logfire-us.pydantic.dev/prayash/hpc4llm\https://logfire-us.pydantic.dev/prayash/hpc4llm]8;;\


In [12]:
print(os.getcwd())

/home/rag/testing/CS5914-High-Performance-Code-Generation-Using-LLMs


In [2]:
os.environ["PATH"] = "/usr/local/cuda-12.4/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.4/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

nvcc_path = shutil.which("nvcc")
if nvcc_path is None:
    print("[ERROR] nvcc not found in PATH. Please ensure that nvcc is installed and its directory is added to the PATH environment variable.")
else:
    print("nvcc found at:", nvcc_path)

nvcc found at: /usr/local/cuda-12.4/bin/nvcc


In [3]:
def clean_generated_code(text: str) -> str:
    """Remove markdown code fences from generated code."""
    cleaned = text.strip()
    if cleaned.startswith("```cuda"):
        lines = cleaned.splitlines()
        if lines[0].strip().startswith("```cuda"):
            lines = lines[1:]
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        cleaned = "\n".join(lines).strip()
    elif cleaned.startswith("```"):
        lines = cleaned.splitlines()
        if lines[0].strip().startswith("```"):
            lines = lines[1:]
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        cleaned = "\n".join(lines).strip()
    return cleaned

In [4]:
prompt = (
    "You are an expert CUDA code generator. Generate a complete and valid CUDA program that launches a kernel where each thread prints:\n"
    "\"Hello from CUDA thread X!\" (where X is the thread index).\n"
    "The program must be self-contained, include a main() function, and be compilable with nvcc (version 12.4, Build cuda_12.4.r12.4/compiler.34097967_0).\n\n"
    "Output must follow this exact format, with no additional commentary or instructions:\n\n"
    "```cuda\n"
    "// Your complete CUDA code here\n"
    "```\n\n"
    "Output only the code block as shown above."
)

In [8]:
generated_dir = Path("data/generated_cuda/integration_check")
generated_dir.mkdir(parents=True, exist_ok=True)

agents = {
    # OpenAI models
    "gpt-4o-mini": Agent(model=OpenAIModel("gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))),
    "o1-mini": Agent(model=OpenAIModel("o1-mini", api_key=os.getenv("OPENAI_API_KEY"))),
    
    # Anthropic models
    "claude-3-5-sonnet-latest": Agent(model=AnthropicModel("claude-3-5-sonnet-latest", api_key=os.getenv("ANTHROPIC_API_KEY"))),
    "claude-3-5-haiku-latest": Agent(model=AnthropicModel("claude-3-5-haiku-latest", api_key=os.getenv("ANTHROPIC_API_KEY"))),
    
    # Gemini models
    "gemini-1.5-flash": Agent(model=GeminiModel("gemini-1.5-flash", api_key=os.getenv("GEMINI_API_KEY"))),
    #"gemini-2.0-flash": Agent(model=GeminiModel("gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY"))),

    # Opensource models
    "llama-3.3-70b-versatile": Agent(model=GroqModel("llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))),
    "qwen-2.5-32b": Agent(model=GroqModel("qwen-2.5-32b", api_key=os.getenv("GROQ_API_KEY"))),
    #"deepseek-r1-distill-qwen-32b": Agent(model=GroqModel("deepseek-r1-distill-qwen-32b", api_key=os.getenv("GROQ_API_KEY")),
}

print("Agents configured:")
for key in agents:
    print(" -", key)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [36]:
results = []

for model_name, agent in agents.items():
    print(f"\n--- Running integration test for {model_name} ---")
    try:
        # Call the agent with our prompt
        result = agent.run_sync(prompt)
        # Extract the generated text (whether structured or plain text)
        code = result.data if hasattr(result, "data") else result
        cleaned_code = clean_generated_code(code)
    except Exception as e:
        print(f"[ERROR] Code generation failed for {model_name}: {e}")
        cleaned_code = ""
    
    # Save the generated CUDA code to a file
    code_file = generated_dir / f"integrationTest_{model_name}.cu"
    with open(code_file, "w") as f:
        f.write(cleaned_code)
    
    # Compile the CUDA code using nvcc
    binary_file = generated_dir / f"integraitonTest_{model_name}.out"
    compile_cmd = ["nvcc", "-O3", str(code_file), "-o", str(binary_file)]
    try:
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True, timeout=30)
        if compile_result.returncode != 0:
            print(f"[ERROR] Compilation failed for {model_name}:\n{compile_result.stderr}")
            exec_output = "Compilation Failed"
            compile_status = "Failure"
        else:
            compile_status = "Success"
            # Run the compiled binary
            run_result = subprocess.run([str(binary_file)], capture_output=True, text=True, timeout=30)
            exec_output = run_result.stdout.strip() if run_result.stdout else run_result.stderr.strip()
    except Exception as e:
        print(f"[ERROR] Exception during compile/run for {model_name}: {e}")
        exec_output = "Error"
        compile_status = "Failure"
    
    # Record the result for this model
    results.append({
        "Model": model_name,
        "Compilation": compile_status,
        "Execution Output": exec_output
    })


--- Running integration test for gpt-4o-mini ---
20:07:25.828 agent run prompt=You are an expert CUDA code generator. Generate a complete and...CUDA code here
```

Output only the code block as shown above.
20:07:25.828   preparing model request params run_step=1
20:07:25.829   model request
20:07:28.612   handle model response
[ERROR] Compilation failed for gpt-4o-mini:
data/generated_cuda/integration_check/integrationTest_gpt-4o-mini.cu(5): error: identifier "printf" is undefined
      printf("Hello from CUDA thread %d!\n", threadId);
      ^

1 error detected in the compilation of "data/generated_cuda/integration_check/integrationTest_gpt-4o-mini.cu".


--- Running integration test for o1-mini ---
20:07:28.841 agent run prompt=You are an expert CUDA code generator. Generate a complete and...CUDA code here
```

Output only the code block as shown above.
20:07:28.842   preparing model request params run_step=1
20:07:28.842   model request
20:07:31.737   handle model response

--- Run

In [29]:
results_csv = Path("data/test_data/integration_results1.csv")
results_csv.parent.mkdir(parents=True, exist_ok=True)
with open(results_csv, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Model", "Compilation", "Execution Output"])
    writer.writeheader()
    writer.writerows(results)
print(f"\nResults saved to {results_csv}")


Results saved to data/test_data/integration_results1.csv


In [30]:
df = pd.read_csv(results_csv)

df["Execution Output"] = df["Execution Output"].apply(
    lambda x: "\n".join(str(x).split("\n")[:3] + ["..."] + str(x).split("\n")[-3:]) 
    if len(str(x).split("\n")) > 6 else x
)

# Display the updated table with summarized output
gt_table = GT(df)
gt_table.show()

Model,Compilation,Execution Output
gpt-4o-mini,Success,Hello from CUDA thread 0! Hello from CUDA thread 1! Hello from CUDA thread 2! ... Hello from CUDA thread 7! Hello from CUDA thread 8! Hello from CUDA thread 9!
o1-mini,Success,Hello from CUDA thread 160! Hello from CUDA thread 161! Hello from CUDA thread 162! ... Hello from CUDA thread 29! Hello from CUDA thread 30! Hello from CUDA thread 31!
claude-3-5-sonnet-latest,Success,Hello from CUDA thread 0! Hello from CUDA thread 1! Hello from CUDA thread 2! ... Hello from CUDA thread 29! Hello from CUDA thread 30! Hello from CUDA thread 31!
claude-3-5-haiku-latest,Success,Hello from CUDA thread 0! Hello from CUDA thread 1! Hello from CUDA thread 2! ... Hello from CUDA thread 9! Hello from CUDA thread 10! Hello from CUDA thread 11!
gemini-1.5-flash,Failure,Compilation Failed
llama-3.3-70b-versatile,Failure,Compilation Failed
qwen-2.5-32b,Success,Hello from CUDA thread 192! Hello from CUDA thread 193! Hello from CUDA thread 194! ... Hello from CUDA thread 253! Hello from CUDA thread 254! Hello from CUDA thread 255!
deepseek-r1-distill-qwen-32b,Failure,Compilation Failed
